# Import packages

In [113]:
import requests
import json
import pandas as pd
import numpy as np
from io import StringIO
from bs4 import BeautifulSoup as bs

from datetime import date
import covidcast

import geopandas as gpd
import sklearn
from sklearn.impute import KNNImputer
pd.set_option('display.max_columns', 100)

In [114]:
url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/mask-use/mask-use-by-county.csv'
s = requests.get(url).text
nymask = pd.read_csv(StringIO(s))

In [115]:
nymask.COUNTYFP = nymask.COUNTYFP.astype(str)
nymask.COUNTYFP = np.where(nymask['COUNTYFP'].str.len() == 4, '0' + nymask.COUNTYFP, nymask.COUNTYFP) 

In [116]:
url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
s = requests.get(url).text
df6 = pd.read_csv(StringIO(s))
print(df6.shape)
print(df6.columns)
# df6.head()

(897801, 6)
Index(['date', 'county', 'state', 'fips', 'cases', 'deaths'], dtype='object')


In [117]:
mask_ind = covidcast.signal("fb-survey", "smoothed_wearing_mask",
                        date(2020, 10, 1), date(2020, 11, 4),
                        "county")

In [118]:
mask_fip = mask_ind.loc[~mask_ind.geo_value.str.endswith('000')]

In [119]:
print(mask_ind.geo_value.value_counts().shape)
mask_fip.geo_value.value_counts().shape

(640,)


(592,)

In [120]:
mask_ind_means = pd.DataFrame(mask_fip.groupby(['geo_value'])['value'].mean()).reset_index()
mask_ind_means.columns = ['FIPS', 'ind_mask']
mask_ind_means

,FIPS,ind_mask
0,01003,75.669396
1,01069,84.805330
2,01073,88.936083
3,01089,89.685856
4,01097,82.273652
...,...,...
587,55133,85.785520
588,55139,88.268850
589,55141,82.577264
590,56021,72.517789


In [121]:
election = pd.read_csv('data/county_statistics.csv', index_col="Unnamed: 0")

In [122]:
election = election[election.county != 'Unassigned']
election['county'] = election['county'] + ', ' + election['state']
# election.drop(['state'], axis=1, inplace=True)

In [123]:
to_keep = ['county', 'total_votes20' , 'percentage20_Joe_Biden']
election = election.dropna(subset=['votes20_Joe_Biden'])

In [124]:
merge_elec = election[to_keep]

In [125]:
print(merge_elec.shape)
# election.columns

(4633, 3)


In [126]:
geo_df = gpd.read_file('https://opendata.arcgis.com/datasets/4cb598ae041348fb92270f102a6783cb_0.geojson')


In [127]:
geo_df = geo_df[~geo_df.Countyname.str.contains("Out of")]
geo_df = geo_df[~geo_df.Countyname.str.contains("Unassigned")]
geo_df = geo_df[~geo_df.ST_Name.str.contains("Puerto Rico")]
geo_df.drop(geo_df.tail(7).index, inplace=True)
geo_df['county'] = geo_df['Countyname'] + ', ' + geo_df['ST_Abbr']

geo_df.drop(columns=['Day_1', 'Day_2', 'Day_3', 'Day_4', 'Day_5', 
                     'Day_6', 'Day_7', 'Day_8', 'Day_9', 'Day_10', 
                     'Day_11', 'Day_12', 'Day_13', 'Day_14', 
                     'OBJECTID', 'ST_ID', 'PCTPOVALL_', 'Med_HH_Inc', 
                     'State_Fata', 'DateChecke', 'url', 'Thumbnail', 
                     'State_Conf', 'State_Deat', 'State_Reco', 'State_Test', 
                     'TotalPop', 'NonHispWhP', 'BlackPop', 'AmIndop', 'PacIslPop', 
                     'OtherPop', 'TwoMorPop', 'HispPop', 'NonHisp', 'Age_85', 
                     'Age_80_84', 'Age_75_79', 'Age_70_74', 'Age_65_69', 
                     'Agetotal', 'AsianPop', 'Countyname', 'ST_Name'], 
            inplace=True)


In [128]:
geo_df.shape

(3142, 45)

In [129]:
merge1 = geo_df.merge(mask_ind_means, how = 'left', on = 'FIPS')

In [130]:
merge1['FIPS'] = to_impute_df.FIPS.astype('int64')

impute_df = to_impute_df.drop(['Inpat_Occ', 'ICU_Occ', 'ST_Abbr', 'EM_type', 'EM_date', 'EM_notes', 'geometry'], axis=1)

In [131]:
knn_imputer = KNNImputer(n_neighbors=6)

In [132]:
imputed = knn_imputer.fit_transform(impute_df.drop(['county'], axis=1))

In [133]:
mask_imputed = pd.DataFrame(imputed, columns = impute_df.drop(['county'], axis=1).columns)['ind_mask']

In [135]:
merge1['ind_mask'] = mask_imputed

In [136]:
elec_geo_df = merge1.merge(merge_elec, on = 'county',how='left', sort=True)
# elec_geo_df = elec_geo_df.merge(mask_ind_means, how = 'left', on = 'FIPS')
elec_geo_df.percentage20_Joe_Biden.isna().sum()

94